# PIT Summary

# Purpose
There has been a lot done in the parameter identification techniques (PIT) in this project, this notebook is a summary.

# Setup

In [ ]:
# %load imports.py
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

#import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl
#import vessel_manoeuvring_models.models.nonlinear_martin_vmm as vmm
#import vessel_manoeuvring_models.nonlinear_martin_vmm_equations as eq

#import vessel_manoeuvring_models.models.linear_vmm as vmm
import vessel_manoeuvring_models.linear_vmm_equations as eq

#import vessel_manoeuvring_models.models.linear_vmm as model
from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

In [ ]:
Math(vlatex(eq.X_eq))

In [ ]:
Math(vlatex(eq.Y_eq))

In [ ]:
Math(vlatex(eq.N_eq))

In [ ]:
Math(vlatex(eq.X_eq.rhs-eq.X_eq.lhs))

In [ ]:
Math(vlatex(eq.Y_eq.rhs-eq.Y_eq.lhs))

In [ ]:
Math(vlatex(eq.N_eq.rhs-eq.N_eq.lhs))

## Load test

In [ ]:
#id=22773
#id=22616
id=22774
#id=22770


df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman')
df.index = df.index.total_seconds()
df = df.iloc[0:-100].copy()
df.index-=df.index[0]
df['t'] = df.index
df.sort_index(inplace=True)
df['-delta'] = -df['delta']
df['V'] = np.sqrt(df['u']**2 + df['v']**2)

df['thrust'] = df['Prop/PS/Thrust'] + df['Prop/SB/Thrust']
df['U'] = df['V']
df['beta'] = -np.arctan2(df['v'],df['u'])

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);
df.plot(y='u')

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2
#I_z_=839.725

ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        #'x_G' : meta_data.lcg,  # motions are expressed at CG
        'x_G' : 0,  # motions are expressed at CG
        
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship
ship_parameters_prime = ps.prime(ship_parameters)

In [ ]:
I_z_+m_*meta_data.lcg**2  # Steiner rule...

In [ ]:
I_z_

In [ ]:
ship_parameters

In [ ]:
ship_parameters_prime

## Prime system

In [ ]:
interesting = ['x0','y0','psi','u','v','r','u1d','v1d','r1d','U','t','delta','thrust','beta']
df_prime = ps.prime(df[interesting], U=df['U'])
df_prime.set_index('t', inplace=True)

In [ ]:
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df_prime, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],
           x_dataset='x0', y_dataset='y0',  psi_dataset='psi', ax=ax);

df_prime.plot(y='u')

In [ ]:
df.index

In [ ]:
df_prime.index

In [ ]:
t_ = np.array([0,1,2])
U_ = np.array([1,2,2])
t_prime = ps._prime(t_, unit='time', U=U_)
t_prime

In [ ]:
ps._unprime(t_prime, unit='time', U=U_)

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

df_parameters.loc['Ydelta','brix_prime'] = 0.0004  # Just guessing
df_parameters.loc['Ndelta','brix_prime'] = -df_parameters.loc['Ydelta','brix_prime']/4  # Just guessing

df_parameters['brix_prime'].fillna(0, inplace=True)
#df_parameters['brix_SI'].fillna(0, inplace=True)



## Simulate with Brix

In [ ]:
X_eq = eq.X_eq.copy()
Y_eq = eq.Y_eq.copy()
N_eq = eq.N_eq.copy()


subs=[
    #(x_G,0),
    (eq.p.Xvdot,0),
    (eq.p.Xrdot,0),
    (eq.p.Yudot,0),
    (eq.p.Yrdot,0),
    (eq.p.Nudot,0),
    (eq.p.Nvdot,0),   
]

X_eq = X_eq.subs(subs)
Y_eq = Y_eq.subs(subs)
N_eq = N_eq.subs(subs)


eqs = [X_eq, Y_eq, N_eq]
solution = sp.solve(eqs, u1d, v1d, r1d, dict=True)

## Decouple the equations:
u1d_eq = sp.Eq(u1d, solution[0][u1d]) 
v1d_eq = sp.Eq(v1d, solution[0][v1d]) 
r1d_eq = sp.Eq(r1d, solution[0][r1d])


## Lambdify:
subs = {value:key for key,value in eq.p.items()}
u1d_lambda = lambdify(u1d_eq.subs(subs).rhs)
v1d_lambda = lambdify(v1d_eq.subs(subs).rhs)
r1d_lambda = lambdify(r1d_eq.subs(subs).rhs)





In [ ]:
from scipy.spatial.transform import Rotation as R

def step(t, states, parameters, ship_parameters, control):

    u,v,r,x0,y0,psi = states
    states_dict = {
        'u':u,
        'v':v,
        'r':r,
        'x0':x0,
        'y0':y0,
        'psi':psi,
        
        }
    
    inputs = dict(parameters)
    inputs.update(ship_parameters)
    inputs.update(states_dict)
    
    if isinstance(control, pd.DataFrame):
        index = np.argmin(np.array(np.abs(control.index - t)))
        control_ = dict(control.iloc[index])
    else:
        control_ = control
    inputs.update(control_)
    
    inputs['U'] = np.sqrt(u**2 + v**2)  #Instantanious velocity
    u1d = run(function=u1d_lambda, inputs=inputs)
    v1d = run(function=v1d_lambda, inputs=inputs)
    r1d = run(function=r1d_lambda, inputs=inputs)
    rotation = R.from_euler('z', psi, degrees=False)
    w = 0
    velocities = rotation.apply([u,v,w])
    x01d = velocities[0]
    y01d = velocities[1]
    psi1d = r    
    dstates = [
        u1d,
        v1d,
        r1d,
        x01d,
        y01d,
        psi1d,
    ]    
    return dstates

In [ ]:
fig,ax=plt.subplots()
df_prime.plot(y='delta', ax=ax)
df_cut_prime = df_prime.iloc[2000:12000]
df_cut_prime.plot(y='delta', ax=ax, style='--', label='cut')

In [ ]:
def simulate(df_, parameters, ship_parameters):
    
    t = df_.index
    t_span = [t.min(),t.max()]
    t_eval = np.linspace(t.min(),t.max(),len(t))
    
    #control = df_[['delta','thrust']]
    control = {
        'delta': df_[['delta']].mean()
    }
    
    df_0 = df_.iloc[0:100].median(axis=0)
    y0 = {
        'u' : df_0['u'], 
        'v' : df_0['v'],
        'r' : df_0['r'],
        'x0' : df_0['x0'],
        'y0' : df_0['y0'],
        'psi' : df_0['psi']
        }
    
    solution = solve_ivp(fun=step, t_span=t_span, y0=list(y0.values()), t_eval=t_eval, 
                args=(parameters, ship_parameters, control))
    
    columns = list(y0.keys())
    df_result = pd.DataFrame(data=solution.y.T, columns=columns)
    df_result.index=t[0:len(df_result)]
    
    df_result['beta'] = -np.arctan2(df_result['v'],df_result['u'])
    
    return solution, df_result

In [ ]:
solution, df_result_brix = simulate(df_cut_prime, parameters = df_parameters['brix_prime'], ship_parameters=ship_parameters_prime)

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df_cut_prime, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='model test')
track_plot(df=df_result_brix, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='simulation', color='green')
ax.legend()


for key in df_result_brix:
    fig,ax = plt.subplots()
    df_cut_prime.plot(y=key, label='model test', ax=ax)
    df_result_brix.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)

## Back to SI

In [ ]:
fig,ax=plt.subplots()
ax.plot(df.index,df_prime.index)

In [ ]:
U_ = ship_parameters['L']*df_prime.index/df.index
df_unprime = ps.unprime(df_prime, U=U_)
df_unprime.index = ps._unprime(df_prime.index,unit='time',U=U_)

In [ ]:
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);
track_plot(df=df_unprime, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

fig,ax=plt.subplots()
df.plot(y='u',ax=ax)
df_unprime.plot(y='u', style='--', ax=ax)

fig,ax=plt.subplots()
df.plot(y='v',ax=ax)
df_unprime.plot(y='v', style='--', ax=ax)

# VCT regression

## Load VCT data

In [ ]:
df_VCT_all = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT_all.head()

In [ ]:
df_VCT = df_VCT_all.groupby(by=['model_name']).get_group('V2_5_MDL_modelScale')

In [ ]:
df_VCT['test type'].unique()

# Subtract the resistance

In [ ]:
df_resistance = df_VCT.groupby(by='test type').get_group('resistance')

X = df_resistance[['u','fx']].copy()
X['u**2'] = X['u']**2
y = X.pop('fx')

model_resistance = sm.OLS(y,X)
results_resistance = model_resistance.fit()

X_pred = pd.DataFrame()
X_pred['u'] = np.linspace(X['u'].min(), X['u'].max(), 20)
X_pred['u**2'] = X_pred['u']**2
X_pred['fx'] = results_resistance.predict(X_pred)

fig,ax=plt.subplots()
df_resistance.plot(x='u', y='fx', style='.', ax=ax)
X_pred.plot(x='u', y='fx', style='--', ax=ax);


In [ ]:
df_VCT_0_resistance = df_VCT.copy()
df_VCT_0_resistance['u**2'] = df_VCT_0_resistance['u']**2
df_VCT_0_resistance['fx']-= results_resistance.predict(df_VCT_0_resistance[['u','u**2']])

## VCT to prime system

In [ ]:
interesting = [
    'u',
    'v',
    'r',
    'delta',
    'fx',
    'fy',
    'mz',
    'thrust',
]
df_VCT_prime = ps_ship.prime(df_VCT_0_resistance[interesting], U=df_VCT_0_resistance['V'])

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred_vct(X,y,results, label):
    
    display(results.summary())
    
    X_ = X.copy()
    X_['y'] = y
    X_.sort_values(by='y', inplace=True)
        
    y_ = X_.pop('y')
    
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    #ax.plot(X_.index,y_, label='Numerical gradient from model test')
    #ax.plot(X_.index,y_pred, '--', label='OLS')
    
    ax.plot(y_,y_pred, '.')
    ax.plot([y_.min(),y_.max()], [y_.min(),y_.max()], 'r-')
    
    ax.set_ylabel(f'{label} (prediction)')    
    ax.set_xlabel(label)
    
    ax.fill_between(y_, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5% confidence')
    ax.legend();

## N

In [ ]:
eq.N_qs_eq

In [ ]:
label = sp.symbols('N_qs')
N_eq_ = eq.N_qs_eq.subs(N_qs,label)
diff_eq_N = regression.DiffEqToMatrix(ode=N_eq_, label=label, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation))

In [ ]:
X = diff_eq_N.calculate_features(data=df_VCT_prime)
y = diff_eq_N.calculate_label(y=df_VCT_prime['mz'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred_vct(X=X,y=y,results=results_N, label=r'$N$')

## Y

In [ ]:
eq.Y_qs_eq

In [ ]:
label = sp.symbols('Y_qs')
Y_eq_ = eq.Y_qs_eq.subs(Y_qs,label)
diff_eq_Y = regression.DiffEqToMatrix(ode=Y_eq_, label=label, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_Y.acceleration_equation))

In [ ]:
X = diff_eq_Y.calculate_features(data=df_VCT_prime)
y = diff_eq_Y.calculate_label(y=df_VCT_prime['fy'])

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred_vct(X=X,y=y,results=results_Y, label=r'$Y$')

## X

In [ ]:
eq.X_qs_eq

In [ ]:
label = sp.symbols('X_qs')
X_eq_ = eq.X_qs_eq.subs(X_qs,label)
diff_eq_X = regression.DiffEqToMatrix(ode=X_eq_, label=label, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_X.acceleration_equation))

In [ ]:
X = diff_eq_X.calculate_features(data=df_VCT_prime)
y = diff_eq_X.calculate_label(y=df_VCT_prime['fx'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred_vct(X=X,y=y,results=results_X, label=r'$X$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'coeff':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['prime'] = df_parameters_all['regressed'].combine_first(df_parameters_all['brix_prime'])  # prefer regressed

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)

mask = ((df_parameters_all['brix_prime']!=0) |
        (pd.notnull(df_parameters_all['regressed'])))
                
df_parameters_all_plot = df_parameters_all.loc[mask]
df_parameters_all_plot.plot.bar(y=['brix_prime','regressed'], ax=ax);

## Simulate

In [ ]:
solution, df_result_VCT = simulate(df_cut_prime, parameters = df_parameters_all['prime'], ship_parameters=ship_parameters_prime)

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df_cut_prime, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='model test')
track_plot(df=df_result_VCT, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='simulation', color='green')
ax.legend()


for key in df_result_VCT:
    fig,ax = plt.subplots()
    df_cut_prime.plot(y=key, label='model test', ax=ax)
    df_result_VCT.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)

# Time series PIT

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X
    y_ = y
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    ax.plot(X_.index,y_, label='Numerical gradient from model test')
    ax.plot(X_.index,y_pred, '--', label='OLS')
    ax.set_ylabel(label)
    
    ax.fill_between(X_.index, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5\% confidence')
    ax.legend();

## N

In [ ]:
N_eq_ = N_eq.copy()
N_eq_ = N_eq_.subs([
    (x_G,0),  # Assuming or moving to CG=0
#    #(I_z,1),  # Removing inertia
#    #(eq.p.Nrdot,0),  # Removing added mass
#    #(eq.p.Nvdot,0),  # Removing added mass
#    #(eq.p.Nudot,0),  # Removing added mass
#    
])  

solution = sp.solve(N_eq_,r1d)[0]
inertia_ = (I_z-eq.p.Nrdot)
N_eq_ = sp.Eq(r1d*inertia_, solution*inertia_)


In [ ]:
Math(vlatex(N_eq_))

In [ ]:
label_N = N_eq_.lhs
diff_eq_N = regression.DiffEqToMatrix(ode=N_eq_, label=label_N, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation))

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation_x))

In [ ]:
Math(vlatex(diff_eq_N.eq_y))

In [ ]:
diff_eq_N.eq_beta

In [ ]:
Math(vlatex(diff_eq_N.eq_X))

In [ ]:
diff_eq_N.y_lambda

In [ ]:
X = diff_eq_N.calculate_features(data=df_prime)
y = run(function=diff_eq_N.y_lambda, inputs=df_prime, **ship_parameters_prime, **df_parameters_all['brix_prime'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$%s$' % vlatex(label_N))

## Y

In [ ]:
Y_eq_ = Y_eq.copy()
Y_eq_ = Y_eq.subs([
    (x_G,0),  # Assuming or moving to CG=0
#    #(I_z,1),  # Removing inertia
#    #(eq.p.Nrdot,0),  # Removing added mass
#    #(eq.p.Nvdot,0),  # Removing added mass
#    #(eq.p.Nudot,0),  # Removing added mass
#    
])  

solution = sp.solve(Y_eq_,v1d)[0]
inertia_ = (eq.p.Yvdot-m)
Y_eq_ = sp.simplify(sp.Eq(v1d*inertia_-U*m*r, solution*inertia_-U*m*r))

Math(vlatex(Y_eq_))

In [ ]:
label_Y = Y_eq_.rhs
diff_eq_Y = regression.DiffEqToMatrix(ode=Y_eq_, label=label_Y, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_Y.calculate_features(data=df_prime)
y = run(function=diff_eq_Y.y_lambda, inputs=df_prime, **ship_parameters_prime, **df_parameters_all['brix_prime'])

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$%s$' % vlatex(label_Y))

## X

In [ ]:
X_eq_ = X_eq.copy()
X_eq_ = X_eq_.subs([
    (x_G,0),  # Assuming or moving to CG=0
#    #(I_z,1),  # Removing inertia
#    #(eq.p.Nrdot,0),  # Removing added mass
#    #(eq.p.Nvdot,0),  # Removing added mass
#    #(eq.p.Nudot,0),  # Removing added mass
#    
])  

solution = sp.solve(X_eq_,u1d)[0]
inertia_ = m-eq.p.Xudot
X_eq_ = sp.simplify(sp.Eq(u1d*inertia_, solution*inertia_))

Math(vlatex(X_eq_))

In [ ]:
label_X = X_eq_.lhs
diff_eq_X = regression.DiffEqToMatrix(ode=X_eq_, label=label_X, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_X.calculate_features(data=df_prime)
y = run(function=diff_eq_X.y_lambda, inputs=df_prime, **ship_parameters_prime, **df_parameters_all['brix_prime'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$%s$' % vlatex(label_X))

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

## Add regressed parameters

In [ ]:
results = pd.concat([results_summary_X, results_summary_Y, results_summary_N],axis=0)
df_parameters_all['PIT'] = results['coeff']
df_parameters_all['PIT'] = df_parameters_all['PIT'].combine_first(df_parameters_all['brix_prime'])  # prefer regressed

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)

mask = ((df_parameters_all['brix_prime']!=0) |
        (pd.notnull(df_parameters_all['regressed'])) | 
        (df_parameters_all['PIT']!=0)
        
       )
                
df_parameters_all_plot = df_parameters_all.loc[mask]
df_parameters_all_plot.plot.bar(y=['brix_prime','regressed','PIT'], ax=ax);

## Simulate

In [ ]:
parameters = df_parameters_all['PIT'].copy()
#parameters['Xv']=0
#parameters['Xr']=0
#parameters['Xu']=0
#parameters['Xdelta']=0
#parameters['Nv']*=-1



solution, df_result_PIT = simulate(df_cut_prime, parameters = parameters, ship_parameters=ship_parameters_prime)

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df_cut_prime, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='model test')
track_plot(df=df_result_PIT, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],ax=ax, label='simulation', color='green')
ax.legend()


for key in df_result_PIT:
    fig,ax = plt.subplots()
    df_cut_prime.plot(y=key, label='model test', ax=ax)
    df_result_PIT.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)